# Coordinte transformation from CRF to TRF

CRF: Celestial Reference Frame, Earth Centered Inertial (ECI) coordinates

TRF: Terrestrial Reference Frame, Earth Centeres Earth Fixed (ECEF) coordinates

Steps of the transformation (the order is important!):
 - the precession of the axis of the Earth (in the Celestial Frame)
 - the nutation of the axis of the Earth (in the Celestial Frame)
 - the rotation of the Earth
 - the pole movement, the movenemt of the axis of the Earh (in the Terrestrial Frame)

In [2]:
# import modules
from math import pi, sin, cos
import numpy as np
from datetime import datetime, timedelta


#helper functions for the rotations

def rx(angle):
    """Rotate around X axis with angle [rad]"""
    return np.array([[1,           0,          0],
                     [0,  cos(angle), sin(angle)],
                     [0, -sin(angle), cos(angle)]])

def ry(angle):
    """Rotate around Y axis with angle [rad]"""
    return np.array([[ cos(angle),  0,  sin(angle)],
                     [          0,  1,           0],
                     [-sin(angle),  0,  cos(angle)]])

def rz(angle):
    """Rotate around Z axis with angle [rad]"""
    return np.array([[ cos(angle),  sin(angle),  0],
                     [-sin(angle),  cos(angle),  0],
                     [          0,           0,  1]])

#input data
#timestamp
year=2019
month=9
day=24
hour=14
minute=17
second=25.1242
#CRF coordinates of the point
x_crf=8405919.291
y_crf=-22918795.321
z_crf=10383353.023
#pole coordinates [rad] and the UT1-UTC difference
x_pole=0.2022*(pi/(180*3600))
y_pole=0.3172*(pi/(180*3600))
tmdiff=-0.15122

The scheme of the transformation:

$$x_{TRF}=R^{M}R^{S}R^{N}R^{P}x_{CRF}$$

Because the coordinates are in column vectors, the multiplication reads from right to left. Each transfrmation is an rotation aroud the centre of the Earth. The rotation matrices:

|notation|transformation|
|-----|-----|
|$R^{P}$|precession|
|$R^{N}$|notation|
|$R^{S}$|rotation|
|$R^{M}$|pole movement|

The first is the precession:

$$R^{P}=R_{Z}\left(-z\right)\cdot R_{Y}\left(\vartheta\right)\cdot R_{Z}\left(-\zeta\right)$$

the angles of the rotations can be calulated by these polinoms:

$$\zeta=2306.2181"T+0.30188"T^{2}+0.017998"T^{3}$$

$$z=2306.2181"T+1.09468"T^{2}+0.018203"T^{3}$$

$$\vartheta=2004.3109"T-0.42665"T^{2}-0.041833"T^{3}$$

the $T$ is the time since the 2000.0 epoch in centuries (36525 days)

In [3]:
#time in centuries since 2000.0
tc0=((datetime(year,month,day)-datetime(2000,1,1)).days-0.5)/36525
tc=tc0+((hour+(minute+second/60)/60)/24)/36525
tcell=((datetime(year,month,day)-datetime(2000,1,1)).days+((hour-12)+(minute+second/60)/60)/24)/36525

#angles in rad
zeta =(2306.2181*tc+0.30188*tc**2+0.017998*tc**3)*(pi/(180*3600))
z    =(2306.2181*tc+1.09468*tc**2+0.018203*tc**3)*(pi/(180*3600))
theta=(2004.3109*tc-0.42665*tc**2-0.041833*tc**3)*(pi/(180*3600))

rot_prec=rz(-z) @ ry(theta) @ rz(-zeta)

tc,tcell, zeta,z,theta,rot_prec

(0.19729214044293608,
 0.1972921404429361,
 0.0022059536098501223,
 0.002206103226605554,
 0.0019170398795102562,
 array([[ 9.99988429e-01, -4.41203847e-03,  1.91703404e-03],
        [ 4.41203847e-03,  9.99990267e-01,  4.22918184e-06],
        [-1.91703404e-03,  4.22889502e-06,  9.99998162e-01]]))

The rotation of the nutation:

$$R^{N}=R_{X}\left(-\left(\varepsilon+\Delta\varepsilon\right)\right)\cdot R_{Z}\left(-\Delta\psi\right)\cdot R_{X}\left(-\varepsilon\right)$$

The $\varepsilon$ can be calculated by this polinom:

$$\varepsilon=23°26'21.448"-46.8150"T-0.00059"T^{2}+0.001813"T^{3}$$

The $\Delta\varepsilon$ and $\Delta\psi$ can be calculates by harmonical series

In [4]:
eps=(23*3600+26*60+21.448-46.8150*tc-0.00059*tc**2+0.001813*tc**3)*(pi/(180*3600))

#                           IAU NUTATION SERIES 2000B (77 TERMS)
#          i, m1, m2, m3, m4, m5,         AA,      BB,    CC,       DD,    EE,    FF
nut_tab=[( 1,  0,  0,  0,  0,  1, -172064161, -174666, 33386, 92052331,  9086, 15377),
         ( 2,  0,  0,  2, -2,  2,  -13170906,   -1675, 13696,  5730336, -3015, -4587),
         ( 3,  0,  0,  2,  0,  2,   -2276413,    -234,  2796,   978459,  -485,  1374),
         ( 4,  0,  0,  0,  0,  2,    2074554,     207,  -698,  -897492,   470,  -291),
         ( 5,  0,  1,  0,  0,  0,    1475877,   -3633, 11817,    73871,  -184, -1924),
         ( 6,  0,  1,  2, -2,  2,    -516821,    1226,  -524,   224386,  -677,  -174),
         ( 7,  1,  0,  0,  0,  0,     711159,      73,  -872,    -6750,     0,   358),
         ( 8,  0,  0,  2,  0,  1,    -387298,    -367,   380,   200728,    18,   318),
         ( 9,  1,  0,  2,  0,  2,    -301461,     -36,   816,   129025,   -63,   367),
         (10,  0, -1,  2, -2,  2,     215829,    -494,   111,   -95929,   299,   132),
         (11,  0,  0,  2, -2,  1,     128227,     137,   181,   -68982,    -9,    39),
         (12, -1,  0,  2,  0,  2,     123457,      11,    19,   -53311,    32,    -4),
         (13, -1,  0,  0,  2,  0,     156994,      10,  -168,    -1235,     0,    82),
         (14,  1,  0,  0,  0,  1,      63110,      63,    27,   -33228,     0,    -9),
         (15, -1,  0,  0,  0,  1,     -57976,     -63,  -189,    31429,     0,   -75),
         (16, -1,  0,  2,  2,  2,     -59641,     -11,   149,    25543,   -11,    66),
         (17,  1,  0,  2,  0,  1,     -51613,     -42,   129,    26366,     0,    78),
         (18, -2,  0,  2,  0,  1,      45893,      50,    31,   -24236,   -10,    20),
         (19,  0,  0,  0,  2,  0,      63384,      11,  -150,    -1220,     0,    29),
         (20,  0,  0,  2,  2,  2,     -38571,      -1,   158,    16452,   -11,    68),
         (21,  0, -2,  2, -2,  2,      32481,       0,     0,   -13870,     0,     0),
         (22, -2,  0,  0,  2,  0,     -47722,       0,   -18,      477,     0,   -25),
         (23,  2,  0,  2,  0,  2,     -31046,      -1,   131,    13238,   -11,    59),
         (24,  1,  0,  2, -2,  2,      28593,       0,    -1,   -12338,    10,    -3),
         (25, -1,  0,  2,  0,  1,      20441,      21,    10,   -10758,     0,    -3),
         (26,  2,  0,  0,  0,  0,      29243,       0,   -74,     -609,     0,    13),
         (27,  0,  0,  2,  0,  0,      25887,       0,   -66,     -550,     0,    11),
         (28,  0,  1,  0,  0,  1,     -14053,     -25,    79,     8551,    -2,   -45),
         (29, -1,  0,  0,  2,  1,      15164,      10,    11,    -8001,     0,    -1),
         (30,  0,  2,  2, -2,  2,     -15794,      72,   -16,     6850,   -42,    -5),
         (31,  0,  0, -2,  2,  0,      21783,       0,    13,     -167,     0,    13),
         (32,  1,  0,  0, -2,  1,     -12873,     -10,   -37,     6953,     0,   -14),
         (33,  0, -1,  0,  0,  1,     -12654,      11,    63,     6415,     0,    26),
         (34, -1,  0,  2,  2,  1,     -10204,       0,    25,     5222,     0,    15),
         (35,  0,  2,  0,  0,  0,      16707,     -85,   -10,      168,    -1,    10),
         (36,  1,  0,  2,  2,  2,      -7691,       0,    44,     3268,     0,    19),
         (37, -2,  0,  2,  0,  0,     -11024,       0,   -14,      104,     0,     2),
         (38,  0,  1,  2,  0,  2,       7566,     -21,   -11,    -3250,     0,    -5),
         (39,  0,  0,  2,  2,  1,      -6637,     -11,    25,     3353,     0,    14),
         (40,  0, -1,  2,  0,  2,      -7141,      21,     8,     3070,     0,     4),
         (41,  0,  0,  0,  2,  1,      -6302,     -11,     2,     3272,     0,     4),
         (42,  1,  0,  2, -2,  1,       5800,      10,     2,    -3045,     0,    -1),
         (43,  2,  0,  2, -2,  2,       6443,       0,    -7,    -2768,     0,    -4),
         (44, -2,  0,  0,  2,  1,      -5774,     -11,   -15,     3041,     0,    -5),
         (45,  2,  0,  2,  0,  1,      -5350,       0,    21,     2695,     0,    12),
         (46,  0, -1,  2, -2,  1,      -4752,     -11,    -3,     2719,     0,    -3),
         (47,  0,  0,  0, -2,  1,      -4940,     -11,   -21,     2720,     0,    -9),
         (48, -1, -1,  0,  2,  0,       7350,       0,    -8,      -51,     0,     4),
         (49,  2,  0,  0, -2,  1,       4065,       0,     6,    -2206,     0,     1),
         (50,  1,  0,  0,  2,  0,       6579,       0,   -24,     -199,     0,     2),
         (51,  0,  1,  2, -2,  1,       3579,       0,     5,    -1900,     0,     1),
         (52,  1, -1,  0,  0,  0,       4725,       0,    -6,      -41,     0,     3),
         (53, -2,  0,  2,  0,  2,      -3075,       0,    -2,     1313,     0,    -1),
         (54,  3,  0,  2,  0,  2,      -2904,       0,    15,     1233,     0,     7),
         (55,  0, -1,  0,  2,  0,       4348,       0,   -10,      -81,     0,     2),
         (56,  1, -1,  2,  0,  2,      -2878,       0,     8,     1232,     0,     4),
         (57,  0,  0,  0,  1,  0,      -4230,       0,     5,      -20,     0,    -2),
         (58, -1, -1,  2,  2,  2,      -2819,       0,     7,     1207,     0,     3),
         (59, -1,  0,  2,  0,  0,      -4056,       0,     5,       40,     0,    -2),
         (60,  0, -1,  2,  2,  2,      -2647,       0,    11,     1129,     0,     5),
         (61, -2,  0,  0,  0,  1,      -2294,       0,   -10,     1266,     0,    -4),
         (62,  1,  1,  2,  0,  2,       2481,       0,    -7,    -1062,     0,    -3),
         (63,  2,  0,  0,  0,  1,       2179,       0,    -2,    -1129,     0,    -2),
         (64, -1,  1,  0,  1,  0,       3276,       0,     1,       -9,     0,     0),
         (65,  1,  1,  0,  0,  0,      -3389,       0,     5,       35,     0,    -2),
         (66,  1,  0,  2,  0,  0,       3339,       0,   -13,     -107,     0,     1),
         (67, -1,  0,  2, -2,  1,      -1987,       0,    -6,     1073,     0,    -2),
         (68,  1,  0,  0,  0,  2,      -1981,       0,     0,      854,     0,     0),
         (69, -1,  0,  0,  1,  0,       4026,       0,  -353,     -553,     0,  -139),
         (70,  0,  0,  2,  1,  2,       1660,       0,    -5,     -710,     0,    -2),
         (71, -1,  0,  2,  4,  2,      -1521,       0,     9,      647,     0,     4),
         (72, -1,  1,  0,  1,  1,       1314,       0,     0,     -700,     0,     0),
         (73,  0, -2,  2, -2,  1,      -1283,       0,     0,      672,     0,     0),
         (74,  1,  0,  2,  2,  1,      -1331,       0,     8,      663,     0,     4),
         (75, -2,  0,  2,  2,  2,       1383,       0,    -2,     -594,     0,    -2),
         (76, -1,  0,  0,  0,  2,       1405,       0,     4,     -610,     0,     2),
         (77,  1,  1,  2, -2,  2,       1290,       0,     0,     -556,     0,     0)]

#mean anomaly of the Moon [rad]
hl=(485868.249036+1717915923.2178*tc+31.8792*tc**2+0.051635*tc**3-0.00024470*tc**4)*(pi/(180*3600))
#mean anomaly of the Sun [rad]
hlp=(1287104.79305+129596581.0481*tc-0.5532*tc**2+0.000136*tc**3-0.00001149*tc**4)*(pi/(180*3600))
#mean argument of the latitude of the Moon [rad]
hf=(335779.526232+1739527262.8478*tc-12.7512*tc**2-0.001037*tc**3+0.00000417*tc**4)*(pi/(180*3600))
#mean elongation of the Moon from the Sun [rad]
hd=(1072260.70369+1602961601.2090*tc-6.3706*tc**2+0.006593*tc**3-0.00003169*tc**4)*(pi/(180*3600))
#mean longitude of the ascending node of the Moon [rad]
hom=(450160.398036-6962890.5431*tc+7.4722*tc**2+0.007702*tc**3-0.00005939*tc**4)*(pi/(180*3600))

delta_psi, delta_eps= 0.0, 0.0 
for element in nut_tab:
    ha=element[1]*hl+element[2]*hlp+element[3]*hf+element[4]*hd+element[5]*hom
    delta_psi+=(element[6]+element[7]*tc)*sin(ha)+element[8]*cos(ha)
    delta_eps+=(element[9]+element[10]*tc)*cos(ha)+element[11]*sin(ha)

delta_psi*=pi/6.48E12
delta_eps*=pi/6.48E12

rot_nut=rx(-(eps+delta_eps)) @ rz(-delta_psi) @ rx(-eps)

eps, delta_eps, delta_psi, rot_nut

(0.40904802566429316,
 -7.512368825138693e-06,
 -8.204727284867876e-05,
 array([[ 9.99999997e-01,  7.52783623e-05, -3.26331600e-05],
        [-7.52786075e-05,  6.83617514e-01, -7.29840454e-01],
        [-3.26325945e-05,  7.29840454e-01,  6.83617517e-01]]))

The rotation matrix for sideral time:

$$R^{S}=R_{Z}\left(\Theta_{0}\right)$$

$$\Theta_{0}=1.0027379093\cdot UT1+\vartheta_{0}+\Delta\psi\cdot\cos\varepsilon$$

$$\vartheta_{0}=24110.54841+8640184.812T_{0}+0.093104T_{0}^{2}-0.0000062T_{0}^{3}$$



In [5]:
ltheta0=24110.54841+8640184.812*tc0+0.093104*tc0**2-0.0000062*tc0**3
utheta0=1.0027379093*(hour*3600+minute*60+second)+ltheta0+delta_psi*cos(eps)*(43200/pi)

rot_sid=rz(utheta0*(pi/43200))

ltheta0, utheta0, rot_sid

(1728610.2553287928,
 1780195.1964609905,
 array([[-0.79356595, -0.60848426,  0.        ],
        [ 0.60848426, -0.79356595,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]))

The rotation of the pole motion:

$$R^{M}=\left[\begin{array}{ccc}
1 & 0 & x_{P}\\
0 & 1 & -y_{P}\\
-x_{P} & y_{P} & 1
\end{array}\right]$$


In [6]:
rot_pm=np.array([[      1,      0,  x_pole],
                 [      0,      1, -y_pole],
                 [-x_pole, y_pole,       1]])

rot_pm

array([[ 1.00000000e+00,  0.00000000e+00,  9.80293263e-07],
       [ 0.00000000e+00,  1.00000000e+00, -1.53782900e-06],
       [-9.80293263e-07,  1.53782900e-06,  1.00000000e+00]])

The coordinate tansformation:

$$x_{TRF}=R^{M}R^{S}R^{N}R^{P}x_{CRF}$$

In [7]:
coord_crf=np.array([x_crf, y_crf, z_crf])

coord_trf=rot_pm @ rot_sid @ rot_nut @ rot_prec @ coord_crf

coord_trf

array([-7661212.30174927,  6402241.52436637, 14153343.40472851])

## References

Bernhard Hofmann-Wellenhof, Herbert Lichtenegger, Elmar Wasle: GNSS — Global Navigation Satellite Systems, Springer-Verlag Wien, 2008. Subsection 2.2.2

http://www.neoprogrammics.com/index.html (Nutation Series)

https://gssc.esa.int/navipedia/index.php/Category:Coordinate_Systems